In [1]:
from pyspark.sql import SparkSession
import os
import configparser

In [2]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

In [4]:
%ls data

log-data.zip  song-data.zip


In [6]:
!unzip data/song-data.zip

Archive:  data/song-data.zip
   creating: song_data/
  inflating: song_data/.DS_Store     
   creating: song_data/A/
  inflating: song_data/A/.DS_Store   
   creating: song_data/A/A/
  inflating: song_data/A/A/.DS_Store  
   creating: song_data/A/A/A/
  inflating: song_data/A/A/A/TRAAAAW128F429D538.json  
  inflating: song_data/A/A/A/TRAAABD128F429CF47.json  
  inflating: song_data/A/A/A/TRAAADZ128F9348C2E.json  
  inflating: song_data/A/A/A/TRAAAEF128F4273421.json  
  inflating: song_data/A/A/A/TRAAAFD128F92F423A.json  
  inflating: song_data/A/A/A/TRAAAMO128F1481E7F.json  
  inflating: song_data/A/A/A/TRAAAMQ128F1460CD3.json  
  inflating: song_data/A/A/A/TRAAAPK128E0786D96.json  
  inflating: song_data/A/A/A/TRAAARJ128F9320760.json  
  inflating: song_data/A/A/A/TRAAAVG12903CFA543.json  
  inflating: song_data/A/A/A/TRAAAVO128F93133D4.json  
   creating: song_data/A/A/B/
  inflating: song_data/A/A/B/TRAABCL128F4286650.json  
  inflating: song_data/A/A/B/TRAABDL12903CAABBA.json  
  i

In [15]:
%cat song_data/A/A/C/TRAACCG128F92E8A55.json

{"num_songs": 1, "artist_id": "AR5KOSW1187FB35FF4", "artist_latitude": 49.80388, "artist_longitude": 15.47491, "artist_location": "Dubai UAE", "artist_name": "Elena", "song_id": "SOZCTXZ12AB0182364", "title": "Setanta matins", "duration": 269.58322, "year": 0}

In [17]:
df = spark.read.json("song_data/A/A/C/TRAACCG128F92E8A55.json")

In [18]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [25]:
df = spark.read.json("song_data/A/A/A")

In [23]:
%ls song_data/A/A/A/

TRAAAAW128F429D538.json  TRAAAFD128F92F423A.json  TRAAARJ128F9320760.json
TRAAABD128F429CF47.json  TRAAAMO128F1481E7F.json  TRAAAVG12903CFA543.json
TRAAADZ128F9348C2E.json  TRAAAMQ128F1460CD3.json  TRAAAVO128F93133D4.json
TRAAAEF128F4273421.json  TRAAAPK128E0786D96.json


In [27]:
schema = df.schema

In [29]:
df2= spark.read.json("song_data/A/", schema=schema)

In [31]:
df2.show(5)

+---------+---------------+---------------+----------------+-----------+--------+---------+-------+-----+----+
|artist_id|artist_latitude|artist_location|artist_longitude|artist_name|duration|num_songs|song_id|title|year|
+---------+---------------+---------------+----------------+-----------+--------+---------+-------+-----+----+
+---------+---------------+---------------+----------------+-----------+--------+---------+-------+-----+----+



In [32]:
df.count()

11

In [38]:
import org.apache.hadoop.fs as hadoop_fs

In [43]:
df3 = spark.read.json("song_data/A/A/{A,B,C}")

In [44]:
df3.count()

36

In [45]:
df4 = spark.read.json("song_data/A/{A,B}/{A,B,C}")

In [46]:
df4.count()

71

In [47]:
df5 = spark.read.json("song_data/{A,B,C}/{A,B,C}/{A,B,C}")

In [48]:
df5.count()

71

In [54]:
import os
import re

In [61]:
count=0
for dirpath, dirnames, filenames in os.walk("song_data/"):
    for filename in filenames :
        fi = os.path.join(dirpath, filename)
        if re.match(".*json", filename):
            count += 1
            print("MATCH", end = "\t")
        print(fi)
            
print(count)

song_data/.DS_Store
song_data/A/.DS_Store
song_data/A/A/.DS_Store
MATCH	song_data/A/A/A/TRAAAAW128F429D538.json
MATCH	song_data/A/A/A/TRAAAVG12903CFA543.json
MATCH	song_data/A/A/A/TRAAARJ128F9320760.json
MATCH	song_data/A/A/A/TRAAAPK128E0786D96.json
MATCH	song_data/A/A/A/TRAAABD128F429CF47.json
MATCH	song_data/A/A/A/TRAAAMO128F1481E7F.json
MATCH	song_data/A/A/A/TRAAAMQ128F1460CD3.json
MATCH	song_data/A/A/A/TRAAAVO128F93133D4.json
MATCH	song_data/A/A/A/TRAAAEF128F4273421.json
MATCH	song_data/A/A/A/TRAAAFD128F92F423A.json
MATCH	song_data/A/A/A/TRAAADZ128F9348C2E.json
MATCH	song_data/A/A/B/TRAABNV128F425CEE1.json
MATCH	song_data/A/A/B/TRAABRB128F9306DD5.json
MATCH	song_data/A/A/B/TRAABLR128F423B7E3.json
MATCH	song_data/A/A/B/TRAABDL12903CAABBA.json
MATCH	song_data/A/A/B/TRAABJL12903CDCF1A.json
MATCH	song_data/A/A/B/TRAABVM128F92CA9DC.json
MATCH	song_data/A/A/B/TRAABYN12903CFD305.json
MATCH	song_data/A/A/B/TRAABXG128F9318EBD.json
MATCH	song_data/A/A/B/TRAABCL128F4286650.json
MATCH	song_dat

In [51]:
import glob

In [52]:
glob.glob("song_data/*json")

[]